In [568]:
import keras
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

In [569]:
np.random.seed(seed=1)

# CSV の準備
df = pd.read_csv('after.csv',encoding = "shift-jis",index_col=0)
l = df.values.tolist()

temp = []
for i in l:
    temp.extend(i)

mapNA = map(str, temp)
text = ''.join(mapNA)

In [570]:
def getPrise(seven, kai):
    
    if first(seven,kai):
        print("7合致")
        return True
    elif third(seven,kai):
        print("6合致")
        return True
    elif fourth(seven,kai):
        print("5合致")
        return True
    elif fifth(seven,kai):
        print("4合致ずさーーーーーーーーーーー！")
        return True
    elif kasuri(seven,kai):
        print("3合致")
        return True
    #elif ganbarimasyou(seven,kai):
    #    print("2合致")
    #    return True
    #else:
        #print("スカ")
    #    return False

def first(seven,checkRow):
    checkR = checkRow[:]
    answerSeven = set(checkR) 
    sevenD = set(seven)  
    sevenD &= answerSeven
    
    if len(sevenD) >= 7:
        return True
    else:
        return False

def third(seven,checkRow):
    checkR = checkRow[:]
    answerSeven = set(checkR)
    sevenD = set(seven)
    
    sevenD &= answerSeven
    if len(sevenD) >= 6:
        return True
    else:
        return False

def fourth(seven,checkRow):
    checkR = checkRow[:]
    answerSeven = set(checkR)
    sevenD = set(seven)
    sevenD &= answerSeven

    if len(sevenD) >= 5:
        return True
    else:
        return False

def fifth(seven,checkRow):
    checkR = checkRow[:]
    answerSeven = set(checkR)
    sevenD = set(seven)
    sevenD &= answerSeven
    
    if len(sevenD) >= 4:
        return True
    else:
        return False

def kasuri(seven,checkRow):
    checkR = checkRow[:]
    answerSeven = set(checkR)
    sevenD = set(seven)
    sevenD &= answerSeven
    
    if len(sevenD) >= 3:
        return True
    else:
        return False
    
def ganbarimasyou(seven,checkRow):
    checkR = checkRow[:]
    answerSeven = set(checkR)
    sevenD = set(seven)
    sevenD &= answerSeven
    
    if len(sevenD) >= 2:
        return True
    else:
        return False

In [571]:
# Length of extracted character sequences
maxlen = 7

# We sample a new sequence every `step` characters
step = 7

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 381
Unique characters: 38
Vectorization...


In [572]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop()
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [573]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

import random
import sys

random.seed(1)

history = []

'''    
for epoch in range(1, int(len(text)/7)):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=1111111,
              epochs=1)
'''
   
model.fit(x, y,
              batch_size=7777,
              epochs=60)

Epoch 1/60
381/381 [==============================] - 3s 7ms/step - loss: 3.6358
Epoch 2/60
381/381 [==============================] - 0s 32us/step - loss: 3.6237
Epoch 3/60
381/381 [==============================] - 0s 31us/step - loss: 3.6142
Epoch 4/60
381/381 [==============================] - 0s 32us/step - loss: 3.6050
Epoch 5/60
381/381 [==============================] - 0s 34us/step - loss: 3.5951
Epoch 6/60
381/381 [==============================] - 0s 34us/step - loss: 3.5839
Epoch 7/60
381/381 [==============================] - 0s 34us/step - loss: 3.5706
Epoch 8/60
381/381 [==============================] - 0s 29us/step - loss: 3.5551
Epoch 9/60
381/381 [==============================] - 0s 31us/step - loss: 3.5380
Epoch 10/60
381/381 [==============================] - 0s 34us/step - loss: 3.5221
Epoch 11/60
381/381 [==============================] - 0s 34us/step - loss: 3.5100
Epoch 12/60
381/381 [==============================] - 0s 32us/step - loss: 3.5000
Epoch 13/60
38

In [574]:
# Select a text seed
#start_index = epoch * 7
#generated_text = text[start_index: start_index + maxlen]

preC = []
preB = []
preA = []
buffC = []
buffB = []
buffA = []

tmp_array = np.arange(0.01,5,0.01)
tmp_list = tmp_array.tolist()
#温度調整
for temperature in tmp_list:
    # We generate 7 characters
    hon_generated_text_C = text[-21:-14]
    #print('--- Generating with seed: "' + hon_generated_text_C + '"')
    for i in range(7):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(hon_generated_text_C):
            sampled[0, t, char_indices[char]] = 1.

        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature = 1.0)
        next_char = chars[next_index]
        hon_generated_text_C += next_char
        hon_generated_text_C = hon_generated_text_C[1:]    

        buffC.append(next_char)
        #sys.stdout.write(next_char)
        #sys.stdout.flush()
        
    preC.append(buffC)
    buffC = []

        
    # We generate 7 characters
    hon_generated_text_B = text[-14:-7]
    #print('--- Generating with seed: "' + hon_generated_text_B + '"')
    for i in range(7):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(hon_generated_text_B):
            sampled[0, t, char_indices[char]] = 1.

        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature = 1.0)
        next_char = chars[next_index]
        hon_generated_text_B += next_char
        hon_generated_text_B = hon_generated_text_B[1:]
                
        buffB.append(next_char)
        #sys.stdout.write(next_char)
        #sys.stdout.flush()
        
    preB.append(buffB)
    buffB = []

    
    # We generate 7 characters
    hon_generated_text_A = text[-7:]
    #print('--- Generating with seed: "' + hon_generated_text_A + '"')
    for i in range(7):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(hon_generated_text_A):
            sampled[0, t, char_indices[char]] = 1.

        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature = 1.0)
        next_char = chars[next_index]
        hon_generated_text_A += next_char
        hon_generated_text_A = hon_generated_text_A[1:]
               
        buffA.append(next_char)
        #sys.stdout.write(next_char)
        #sys.stdout.flush()        

    preA.append(buffA)
    buffA = []

In [578]:
numA = 0
for mmA in preA:
    numA += 1
    print(numA, mmA)

1 ['k', '四', 'u', 'k', 'e', 'm', '八']
2 ['a', 'c', 's', 'v', 'r', 'p', 'm']
3 ['r', 'u', 'j', 'r', 'r', 's', 'r']
4 ['n', '七', '一', 'm', '二', 'i', 'u']
5 ['二', 'c', 'v', '六', 'm', '一', '六']
6 ['五', 'u', '四', '零', 's', 'u', '零']
7 ['a', 'u', 'g', 'v', 'o', 'i', '六']
8 ['c', 'j', 'l', 'e', '八', 'j', 'u']
9 ['u', 'v', 'i', 'e', '六', 'o', 'e']
10 ['四', 'p', '零', 'l', 'i', '八', 'm']
11 ['o', 's', '二', '六', 'y', 'e', 'w']
12 ['二', 'c', 'j', 'l', 'l', 'g', 'j']
13 ['二', 'u', '四', '五', '五', 'v', 'w']
14 ['二', 'j', 'm', 'j', '一', 'j', '八']
15 ['二', 'm', 'y', 'l', 'u', '零', '二']
16 ['u', 'q', 'w', 'r', 'g', 'w', 'o']
17 ['c', 'i', 'r', 'c', 'n', 'w', '八']
18 ['二', '一', '二', 'i', 'f', '一', 'a']
19 ['r', '零', 'r', 'j', 'u', 'g', '零']
20 ['六', '七', 'k', 'r', 'u', 's', 'i']
21 ['二', 'g', '一', 'n', '一', '二', '一']
22 ['l', 'c', 'c', '六', 'f', '二', 'f']
23 ['n', 'w', '二', 'c', 'w', 'k', 'f']
24 ['z', '四', 'r', 'g', '七', '零', 'g']
25 ['g', 'w', '四', 'g', 'i', 'x', 'm']
26 ['g', 'm', '二', '三', 'm', 'r', 

In [580]:
numB = 0
for mmB in preB:
    numB += 1
    if getPrise('hevsc未', mmB) == True :
        print(numB, mmB)

3合致
59 ['六', '三', 'h', 'a', 'c', 'z', 's']
3合致
72 ['h', '六', 'j', 'v', '一', 'c', 'f']
3合致
77 ['h', 'h', 'v', 'h', 'u', 't', 'c']
3合致
228 ['h', '四', '零', 'v', 's', 'b', 'z']
3合致
264 ['h', 'i', 'd', 'e', 'd', 'v', 'o']
3合致
278 ['一', '三', 'v', 'h', 'e', '九', 'r']
3合致
297 ['h', '三', '三', 'v', 'c', '四', 'u']
3合致
298 ['h', 'v', '六', 't', 'k', 'e', 'f']
3合致
299 ['h', 'l', 'c', 'w', 'e', 'd', 'c']
3合致
347 ['九', 'f', 'h', 'v', 'u', 'c', 'o']
3合致
496 ['v', 'k', 'k', 'h', 'h', 'v', 's']
3合致
499 ['h', 'j', 'u', 'd', 'e', 's', 'o']


In [581]:
numC = 0
for mmC in preC:
    numC += 1
    if getPrise('七dyh二一z', mmC) == True :
        print(numC, mmC)

3合致
43 ['z', 'j', '二', 'm', 'i', 'y', 'u']
4合致ずさーーーーーーーーーーー！
46 ['z', 'm', '七', '三', 'y', 'h', 'o']
4合致ずさーーーーーーーーーーー！
62 ['七', 's', '零', 'z', 'y', 'h', '五']
3合致
70 ['k', 'z', 'a', 'c', 'y', '二', '四']
4合致ずさーーーーーーーーーーー！
81 ['七', '零', 'u', '九', 'y', '一', 'h']
3合致
85 ['七', '零', 'w', 'y', '二', 'o', '零']
3合致
86 ['z', '零', '零', '三', 'y', '一', 'f']
3合致
104 ['z', 't', 'k', '零', 'd', '一', '四']
3合致
106 ['七', '一', 'b', 'u', '零', '五', 'y']
3合致
139 ['k', '七', '七', 'y', '三', 'h', '五']
3合致
152 ['j', '七', 's', '二', 'x', 'h', '零']
4合致ずさーーーーーーーーーーー！
184 ['七', 'k', 'c', '一', 'y', 'h', 't']
3合致
187 ['a', 'z', 'm', 'p', 'p', 'h', 'y']
3合致
199 ['零', 's', '七', '五', 'y', '二', 'u']
3合致
202 ['z', 's', '二', '六', 'g', 'd', 'z']
3合致
213 ['七', 'k', '二', 'q', 'd', '七', 't']
3合致
238 ['七', 'k', 'm', '七', 'y', 'y', 'h']
3合致
251 ['七', 'c', 'u', '一', 'r', 'h', 'x']
3合致
254 ['k', '七', 'c', '二', '一', 'v', 'n']
4合致ずさーーーーーーーーーーー！
256 ['七', 'z', 'm', 'a', 'h', 'd', 't']
4合致ずさーーーーーーーーーーー！
289 ['s', 'd', '七', 'k', 'y', '四', 'h']

In [579]:
text

'jglqw八四三x一九五t四gkhwb一n八vm九xlw八dupaec五so四ew零gac三e九七u二零九b八六cwd六零七obcxfa八零i九四二o六z三sn七zlk八j七五fupigh零xd八bw五yqjnwiupfeme六四bc末v六九uqxl一ntsvb一uum八ebwto零iy三maodiwhybd八九hky零hsd三pjerakpqufrkpm一六znrjakq八gsl五末ofcz八qxpbzmhfv六二三四wilms七roc九h末九dax三q六w五g二xdfa四sr零yp八t六fekjhg末mnsj四n九sxdvzymsgw零z四bwfk一bo五ch六四obqpw三一fh一gdx末ocd八九f七u三abr七if七ow零beuntbki六exz零九四ckhx七zlqcoxg九hdedwah五tz四二vhxj三pdt六anejvrcwaqd六w零nl二六wdz七八三eptflqghfw二三a五uhem零loi四gwcpjke一八末u二o七djl三hx零二八sdpq六t九do八xwlvdgk三a一y八末orykgq五o四xg五bmt八vm末九q七r三jui五五九四fm末al二skr一t零七wlp二yopmnex一五dqios末六rm九四二dz八yc零wjfnxe六八jg九buid末rbxue七j五a一crj八yca七pt四五ua二k八z九五hpw六八fkgzvj五tcvzju零htrl一i八n九j零dyc四ak六z八crm末inbhn五eayogdz二九eyoqeb零nk二零九st七m八nr四amh零ukiwnvxgxnvmp五ikm三ycsxq五八mv七y六kb五apuwl末八k六a八七一九dhvcb五l六kgokvip末o八zct三sjuvslf零i三qcmh七h六fbg八零零fqutx二六一v零五ld八ix三g零h五f七j六goofdi末gq九h末z八七mthnmyf七w八一in七二ij一四末八te零h七ix八fyh六ocn末zbp五二rbud零pm末p七z四五bre九cha四六rbqao六vkan五八fsmjdpuq七五ol零w八j二zhb七wdexkg七s零ta二wy一o六四二qldcj一gtn零p零cmoqgu五八四tid零mxi四hn二零末xjq七三sy二m七u五三tub二eqiwru四p五五zkqlcm零jb二r一六nru七fl八八hca九